In [2]:
from scipy.stats import nbinom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from IPython.display import display, HTML
from matplotlib.pyplot import figure

In [3]:
# This script will generate a simulated read count file for a disomy and trisomy dataset that can be fed directly into
# DESEQ2. Essentially, a previous RNA-seq dataset is used as the basis for finding mu for each gene. Replicates from 
# Eric are used to sample first from a normal distribution to generate mu, and then the
# dispersion is calculated through parametric fit (y = a + b/x) where a is asymptotic dispersion and b is extra-poisson
# noise. a and b thus act as hyperparameters for our sampling. Additional parameters include the number of replicates in
# a single experiment, the number of experiments (trials), and a scaling factor to increase the depth of the samples


trials=3 # The number of experiments to generate. A new file will be generated with the same parameters for [1:trials)

disomy_disp=.01 # The asymptotic dispersion (hyperparameter a) for the disomy simulations
disomy_extrapoiss=1 # The extra-poisson noise (hyperparameter b) for the disomy simulations
asymptDisp=disomy_disp
extraPois=disomy_extrapoiss

test_array=np.linspace(3,10,8).astype(int) # numpy list for iterating over multiple values of the hyperparameters or replicate/scales

for test_num in test_array: # I just use this to iterate through values I'm interested in testing (dispersion, poisson noise, depth, or replication)
    print(test_num)
    trisomy_disp=.03 # The asymptotic dispersion for trisomy simulation (a)
    trisomy_extrapoiss=5 # The extrapoisson noise for trisomy simulation (b)
    scaling=3 # a scalar to increase depth. Each value of the original mean counts will be multiplied by this to simulate more depth
    replicate_num=test_num # How many replicates to make for a single experiment (e.g., 3 will generate 3 disomic and 3 trisomic datasets)
    for trial_num in range(1,trials):
        # DISOMY SECTION
        # We first read in a real count file generated from biological data
        counts_df = pd.read_csv('/Shares/down/mixed/RNAGRO01132021/data/RNAseqfiles/res_featureCounts_gene_idfull_143138.coverage.csv')
        # First, we'll generate a normal distribution for the mean
        # We'll calculate a mean and variance for each gene from Eric, and sample from that to get our simulated mean
        
        # We also need an annotation df to assess which genes belong to chr21
        annotation_df = pd.read_csv('/Shares/down/mixed/RNAGRO01132021/data/RNAseqfiles/res_featureCounts_gene_idfull_143138.annotation.csv')
        annotation_df=annotation_df[annotation_df['Chr'].str.match('chr21')]
        chr21genes = annotation_df['GeneID']       
        
        # First we average the counts from Eric's columns, and find the variance
        erics_cols = ['Eric_repA_tech1.sorted.bam', 'Eric_repB_tech1.2.sorted.bam','Eric_repC_tech1.sorted.bam']
        counts_df['Eric_average'] = np.ceil(counts_df[erics_cols].mean(axis=1)) 
        counts_df['Eric_variance'] = counts_df[erics_cols].var(axis=1) 
        genenames= counts_df['Unnamed: 0']
        basecounts= counts_df['Eric_repA_tech1.sorted.bam']
        d = {'Gene_id':genenames,'Eric_counts':basecounts}
        simulated_df = pd.DataFrame(d) # This dataframe is what we'll actually be adding to for each replicate

        means=counts_df['Eric_average'] # This is used as input for sampling from a normal distribution
        variances=counts_df['Eric_variance'] # Same for this one
        counts_df['diploidy'] = np.where( # Disomy individuals have the same ploidy for each gene, but I include it as a np.where if you wanted to play with this a bit
            counts_df['Unnamed: 0'].isin(chr21genes),                # Condition (disomy).
            1,  # . 
            1     # What to assign to price when condition is false. 
        )
        # Here are the sims for the D21 individual. We need to modify our numbers for the T21 individual
        sim_name="Elvis"# Just a silly name to save our disomy file to
        
        # We'll just assign p and n for means at or below zero to avoid infinite dispersion
        # Same for samples with 0 variance to prevent infinite p
        # Note that since mean ~ variance in NB, this will only occur with means of 0 anyway, so these'll be discarded when
        # we run DESeq2 
        for j in range(replicate_num):    
            counts_df['Eric_disomy'] = (counts_df['Eric_average'] * counts_df['diploidy'])
            means=counts_df['Eric_disomy']
            simulated_counts=[]
            for i in range(len(means)):
                #newmean=means[i] #Uncomment this if you don't want to sample from a normal first
                newmean=(np.ceil(norm.rvs(size=1,loc=((means[i]+1)*scaling),scale=math.sqrt(variances[i]))))
                if newmean <= 0:
                    newmean=0 # This prevents negative dispersions, as counts should never go below 0
                    dispersion=asymptDisp+(extraPois/1)
                else:
                    dispersion=asymptDisp+(extraPois/newmean)            
                var=newmean+(dispersion*(newmean**2))
                if var == 0:
                    var=1 # This prevents dividing by 0, but won't change the value of p or n
                p=newmean/var
                n=(newmean**2)/((var)-newmean)
                if (p == 0) and (n==0): # We can just assign r as 0 to avoid undefined values in p and n are 0. This only happens if the original mean is 0
                    r = 0
                    simulated_counts.append(r)
                    continue   
                r = nbinom.rvs(n, p, size=1) # sample from negative binomial with parameters n and p
                simulated_counts.append(r[0])
            colname=sim_name+"_rep"+str(j+1)+"_counts"
            simulated_df[colname]=simulated_counts 
        sim_name="EDad"# Just a silly name to save our disomy file to
        
        # We'll just assign p and n for means at or below zero to avoid infinite dispersion
        # Same for samples with 0 variance to prevent infinite p
        # Note that since mean ~ variance in NB, this will only occur with means of 0 anyway, so these'll be discarded when
        # we run DESeq2 
        for j in range(replicate_num):    
            counts_df['Eric_disomy'] = (counts_df['Eric_average'] * counts_df['diploidy'])
            means=counts_df['Eric_disomy']
            simulated_counts=[]
            for i in range(len(means)):
                #newmean=means[i] #Uncomment this if you don't want to sample from a normal first
                newmean=(np.ceil(norm.rvs(size=1,loc=((means[i]+1)*scaling),scale=math.sqrt(variances[i]))))
                if newmean <= 0:
                    newmean=0 # This prevents negative dispersions, as counts should never go below 0
                    dispersion=asymptDisp+(extraPois/1)
                else:
                    dispersion=asymptDisp+(extraPois/newmean)            
                var=newmean+(dispersion*(newmean**2))
                if var == 0:
                    var=1 # This prevents dividing by 0, but won't change the value of p or n
                p=newmean/var
                n=(newmean**2)/((var)-newmean)
                if (p == 0) and (n==0): # We can just assign r as 0 to avoid undefined values in p and n are 0. This only happens if the original mean is 0
                    r = 0
                    simulated_counts.append(r)
                    continue   
                r = nbinom.rvs(n, p, size=1) # sample from negative binomial with parameters n and p
                simulated_counts.append(r[0])
            colname=sim_name+"_rep"+str(j+1)+"_counts"
            simulated_df[colname]=simulated_counts 

        sim_name="EMom"# Just a silly name to save our disomy file to
        
        # We'll just assign p and n for means at or below zero to avoid infinite dispersion
        # Same for samples with 0 variance to prevent infinite p
        # Note that since mean ~ variance in NB, this will only occur with means of 0 anyway, so these'll be discarded when
        # we run DESeq2 
        for j in range(replicate_num):    
            counts_df['Eric_disomy'] = (counts_df['Eric_average'] * counts_df['diploidy'])
            means=counts_df['Eric_disomy']
            simulated_counts=[]
            for i in range(len(means)):
                #newmean=means[i] #Uncomment this if you don't want to sample from a normal first
                newmean=(np.ceil(norm.rvs(size=1,loc=((means[i]+1)*scaling),scale=math.sqrt(variances[i]))))
                if newmean <= 0:
                    newmean=0 # This prevents negative dispersions, as counts should never go below 0
                    dispersion=asymptDisp+(extraPois/1)
                else:
                    dispersion=asymptDisp+(extraPois/newmean)            
                var=newmean+(dispersion*(newmean**2))
                if var == 0:
                    var=1 # This prevents dividing by 0, but won't change the value of p or ns
                p=newmean/var
                n=(newmean**2)/((var)-newmean)
                if (p == 0) and (n==0): # We can just assign r as 0 to avoid undefined values in p and n are 0. This only happens if the original mean is 0
                    r = 0
                    simulated_counts.append(r)
                    continue   
                r = nbinom.rvs(n, p, size=1) # sample from negative binomial with parameters n and p
                simulated_counts.append(r[0])
            colname=sim_name+"_rep"+str(j+1)+"_counts"
            simulated_df[colname]=simulated_counts 
            
        # This calculates a sum which you should check against DESeq2's size factor normalization if you want to make sure 
        column_number=len(simulated_df.columns)
        norm_sum=simulated_df.iloc[:, column_number-replicate_num:column_number].sum().mean()
        
        
        # TRISOMY SECTION
        annotation_df = pd.read_csv('/Shares/down/mixed/RNAGRO01132021/data/RNAseqfiles/res_featureCounts_gene_idfull_143138.annotation.csv')
        annotation_df=annotation_df[annotation_df['Chr'].str.match('chr21')]
        counts_df = pd.read_csv('/Shares/down/mixed/RNAGRO01132021/data/RNAseqfiles/res_featureCounts_gene_idfull_143138.coverage.csv')

        counts_df['Eric_average'] = np.ceil(counts_df[erics_cols].mean(axis=1)) 
        counts_df['Eric_variance'] = counts_df[erics_cols].var(axis=1)

        chr21genes = annotation_df['GeneID']
        counts_df['ploidy'] = np.where(
            counts_df['Unnamed: 0'].isin(chr21genes),                # Condition (belongs to chr21).
            1.1,  # What to assign to price when condition is true. 
            1     # What to assign to price when condition is false.
        )
        gene_ploidy=counts_df['ploidy']

        # Here are the sims for the T21 individual. We need to modify our numbers for the T21 individual
        sim_name="Eddie" # Silly name for T21 sim. Change as you like!

        for j in range(replicate_num):
            counts_df['Eric_trisomy'] = (counts_df['Eric_average'] * counts_df['ploidy'])
            means=counts_df['Eric_trisomy']
            variances=counts_df['Eric_variance']
            simulated_counts=[]
            for i in range(len(means)):
                if gene_ploidy[i]==1.1: # If the gene is triploid, we'll assign the trisomy dispersion
                    asymptDisp=trisomy_disp
                    extraPois=trisomy_extrapoiss
                else: # Otherwise we assign the original values
                    asymptDisp=disomy_disp
                    extraPois=disomy_extrapoiss
                # Note that this is NOT the way it works in DESeq2, where all gene dispersions are used to inform
                # the parametric fit. We do it here to demonstrate the effect of trisomy genes having higher variance
                # within an individual sample
                newmean=(np.ceil(norm.rvs(size=1,loc=((means[i]+1)*scaling),scale=math.sqrt(variances[i])))) # Scale up mean as desired to represent more depth
                if newmean <= 0:
                    newmean=0
                    dispersion=asymptDisp+(extraPois/1)
                else:
                    dispersion=asymptDisp+(extraPois/newmean)
                var=newmean+(dispersion*(newmean**2))
                if var == 0:
                    var=1
                p=newmean/var
                n=(newmean**2)/((var)-newmean)
                if (p == 0) and (n==0):
                    r = 0
                    simulated_counts.append(r)
                    continue   
                r = nbinom.rvs(n, p, size=1)
                simulated_counts.append(r[0])
            colname=sim_name+"_rep"+str(j+1)+"_counts"
            #norm_factor=(norm_sum/sum(simulated_counts)) # You can re-normalize the counts to be the same depth by uncommenting this
            norm_factor=1 # And commenting this instead
            simulated_counts = [np.ceil(norm_factor*x) for x in simulated_counts]
            simulated_df[colname]=simulated_counts  


        simulated_df = simulated_df.drop(columns=['Eric_counts'])
        filename="/Users/sahu0957/trisomy_normalization/RNAGRO01132021/data/RNAseqfiles/POWERANALYSIS_1.1_finalsims_a"+str(trisomy_disp)+"_p"+str(trisomy_extrapoiss)+"_r"+str(trial_num)+"_s"+str(scaling)+"_n"+str(replicate_num)+".csv"
        simulated_df.to_csv(filename, sep=',')


3
4
5
6
7
8
9
10
